In [1]:
from streamlit_app import *

In [2]:
import datetime
from logging import exception
from typing import List

import gspread
import numpy as np
import pandas as pd
import plotly.express as px
import streamlit as st
from google.oauth2 import service_account
import matplotlib.colors as mcolors

In [2]:
sheet_url = st.secrets["private_gsheets_url"]
df = get_worksheet(sheet_url, 0)

2022-06-24 22:33:24.684 
  command:

    streamlit run /home/wotashu/.cache/pypoetry/virtualenvs/streamlit-cheki-5kyQcRg0-py3.9/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [3]:
cheki_df = get_worksheet(sheet_url, 0)
person_df = get_worksheet(sheet_url, 1)
venue_df = get_worksheet(sheet_url, 2)

In [63]:
cheki_df = get_dates(cheki_df)

In [64]:
cheki_df[cheki_df.isin(["Suzy"]).any(1)]

,note,location,date,name1,name2,name3,name4,name5,name6,name7,name8,month,year
186,,O-East,2019-05-06,Suzy,,,,,,,,5,2019
187,,O-East,2019-05-06,Suzy,,,,,,,,5,2019
188,,O-East,2019-05-06,Suzy,,,,,,,,5,2019
189,,O-East,2019-05-06,Suzy,,,,,,,,5,2019
289,,新宿ReNY,2020-01-03,Suzy,,,,,,,,1,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5585,,Cosmic Lab,2022-06-18,Suzy,,,,,,,,6,2022
5586,,Cosmic Lab,2022-06-18,Suzy,,,,,,,,6,2022
5587,,Cosmic Lab,2022-06-18,Suzy,,,,,,,,6,2022
5588,,Cosmic Lab,2022-06-18,Suzy,,,,,,,,6,2022


In [8]:
name_cols = [i for i in cheki_df.columns if "name" in i]

In [26]:
cheki_array = cheki_df[name_cols].replace("", None).values

In [33]:
from collections import defaultdict
counts = defaultdict(list)
for row in cheki_array:
    updated_row = list(filter(None, set(row)))
    row_size = len(updated_row)
    for name in updated_row:
        counts[name].append(row_size)

In [35]:
from collections import Counter

Counter(counts["Suzy"])

Counter({1: 942, 5: 5, 4: 5, 3: 20, 2: 21})

In [36]:
grouped_counts = defaultdict()
for key, value in counts.items():
    grouped_counts[key] = Counter(value)

In [41]:
names_df = pd.DataFrame(grouped_counts).T.fillna(0)

In [43]:
names_df['total'] = names_df.sum(axis=1)

In [57]:
names_df.sort_values(by="total", ascending=False)[:20][['total', 1, 2, 3, 4, 5, 6, 7, 8]].astype(int)

,total,1,2,3,4,5,6,7,8
Suzy,993,942,21,20,5,5,0,0,0
恵深あむ@あむみほ,754,717,26,2,2,5,2,0,0
七瀬千夏@たまご姫【たまプリ】,310,272,7,19,1,0,3,8,0
瀬乃悠月@恒星宇宙,273,244,23,3,0,2,1,0,0
天音ゆめ@たまご姫【たまプリ】,232,205,5,1,9,1,3,8,0
楠木りほ@天使突抜ニ読ミ,217,194,4,19,0,0,0,0,0
火野快飛@恒星宇宙,133,107,20,3,0,2,1,0,0
中谷亜優@たまご姫【たまプリ】,124,102,1,1,8,1,3,8,0
雅春奈@ユレルランドスケープ,123,106,8,6,3,0,0,0,0
椎名まどか@たまご姫【たまプリ】,123,99,2,1,9,1,3,8,0


In [95]:
def group_cheki_by_name(df):
    name_cols = [col for col in df.columns if "name" in col]
    chekis = []
    for idx, row in df.iterrows():
        name_count = len(list(filter(None, row[name_cols].values)))
        for col in df.columns:
            if (row[col] is not np.nan) and ("name" in col) and (row[col]):
                chekis.append(
                    {
                        "cheki_id": idx,
                        "date": row["date"].date(),
                        "person": row[col],
                        "location": row["location"],
                        "year": row["date"].year,
                        "month": row["date"].month,
                        "name": split_name_group(row[col])[0],
                        "group": split_name_group(row[col])[1],
                        "n_shown": name_count, 
                    }
                )
    return pd.DataFrame(chekis)

In [96]:
export_df = group_cheki_by_name(cheki_df)

In [97]:
export_df[export_df["name"]=="Suzy"].groupby(["name", "n_shown"])['person'].count()

name  n_shown
Suzy  1          942
      2           21
      3           20
      4            5
      5            5
Name: person, dtype: int64

In [115]:
topdown = export_df.groupby(["name", "n_shown"])['person'].count().sort_index(level=0).reset_index()

In [116]:
topdown

,name,n_shown,person
0,AIRI,1,1
1,Aarie,1,1
2,Ale 愛礼,1,4
3,Ally,1,7
4,Ally,2,2
...,...,...,...
533,鳴上綺羅,6,1
534,黒ゑリズ,1,1
535,黒木結愛,1,2
536,黒瀬瑠衣,1,25


In [125]:
wide_df = topdown.pivot(index="name", columns="n_shown", values="person").fillna(0).astype(int)

In [126]:
wide_df['total'] = wide_df.sum(axis=1)

In [129]:
wide_df = wide_df[['total'] + [i for i in range(1,9)]]

In [148]:
totals = wide_df.sort_values(by=["total"], ascending=False).reset_index()

In [155]:
totals.index.name = "yay"

In [159]:
totals

n_shown,name,total,1,2,3,4,5,6,7,8
yay,,,,,,,,,,
0,Suzy,993,942,21,20,5,5,0,0,0
1,恵深あむ,754,717,26,2,2,5,2,0,0
2,七瀬千夏,310,272,7,19,1,0,3,8,0
3,瀬乃悠月,273,244,23,3,0,2,1,0,0
4,天音ゆめ,232,205,5,1,9,1,3,8,0
...,...,...,...,...,...,...,...,...,...,...
383,柊・D・ナナ,1,1,0,0,0,0,0,0,0
384,やぎぬま,1,1,0,0,0,0,0,0,0
385,みゆりん,1,1,0,0,0,0,0,0,0
